In [1]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-67259536-64d4-4b2e-b700-39d94e962a7a;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 445ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/13 01:14:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/13 01:14:14 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/13 01:14:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/13 01:14:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
%run build_adjacency.ipynb

In [8]:
df_epm = readCheckpoint(size = "medium")
df_epm = df_epm.limit(50000)
df_ohe_test = build_adjaceny(df_epm)

 1:25AM UTC on Mar 13, 2023 --- read and cache time: 0.3010730743408203 seconds ---
397
length actIDs: 125
elapsed time: 1.6015713214874268
8f18426d-d87e-479f-802d-d6f49a34a1e1


no birth event for 17 events.
elapsed time 2: 253.75643801689148


In [9]:
df_ohe_test.printSchema()

root
 |-- order: integer (nullable = false)
 |-- parent_objectid: string (nullable = true)
 |-- parent_timestamp: string (nullable = true)
 |-- parent_object: string (nullable = true)
 |-- parent_parent_image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- parent_USER: string (nullable = true)
 |-- child_objectid: string (nullable = true)
 |-- child_timestamp: string (nullable = true)
 |-- child_object: string (nullable = true)
 |-- child_parent_image_path: string (nullable = true)
 |-- child_image_path: string (nullable = true)
 |-- child_USER: string (nullable = true)



In [10]:
df_ohe_test.select(col("child_image_path") == "PING.EXE").count()

+--------------------+
|    child_image_path|
+--------------------+
|\Device\HarddiskV...|
|              System|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\\?\C:\Program Fi...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|            PING.EXE|
+--------------------+



In [11]:
df_ohe_test.select(col("child_image_path") == "PING.EXE").count()

431

In [21]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='parent_object', outputCol='p_obj_numeric')
indexer_fitted = indexer.fit(df_ohe_test)
df_indexed = indexer_fitted.transform(df_ohe_test)

encoder = OneHotEncoder(inputCols=['p_obj_numeric'], outputCols=['p_obj_sparse'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot.select('parent_object','p_obj_sparse').limit(20).show()

+-------------+-------------+
|parent_object| p_obj_sparse|
+-------------+-------------+
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
+-------------+-------------+



In [18]:
df_onehot.printSchema()

root
 |-- order: integer (nullable = false)
 |-- parent_objectid: string (nullable = true)
 |-- parent_timestamp: string (nullable = true)
 |-- parent_object: string (nullable = true)
 |-- parent_parent_image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- parent_USER: string (nullable = true)
 |-- child_objectid: string (nullable = true)
 |-- child_timestamp: string (nullable = true)
 |-- child_object: string (nullable = true)
 |-- child_parent_image_path: string (nullable = true)
 |-- child_image_path: string (nullable = true)
 |-- child_USER: string (nullable = true)
 |-- p_obj_numeric: double (nullable = false)
 |-- p_obj_sparse: vector (nullable = true)



In [22]:
#create sparse vecotrs for child objects
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='child_object', outputCol='c_obj_numeric')
indexer_fitted = indexer.fit(df_onehot)
df_indexed = indexer_fitted.transform(df_onehot)

encoder = OneHotEncoder(inputCols=['c_obj_numeric'], outputCols=['c_obj_sparse'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot = df_onehot.drop('c_obj_numeric', 'p_obj_numeric')
df_onehot.select('p_obj_sparse','c_obj_sparse').limit(20).show()

+-------------+-------------+
| p_obj_sparse| c_obj_sparse|
+-------------+-------------+
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
+-------------+-------------+



In [23]:
#combine sparse vectors and turn into string
from pyspark.ml.feature import VectorAssembler
# Use VectorAssembler to concatenate the two sparse vectors
assembler = VectorAssembler(inputCols=['p_obj_sparse', 'c_obj_sparse'], outputCol="comb_sparse")
df_onehot_comb = assembler.transform(df_onehot)

In [24]:
df_onehot_comb.select('comb_sparse').limit(5).show()

+-------------+
|  comb_sparse|
+-------------+
|[1.0,1.0,0.0]|
|[1.0,1.0,0.0]|
|[1.0,0.0,1.0]|
|[1.0,1.0,0.0]|
|[1.0,1.0,0.0]|
+-------------+



In [25]:
#create a timestamp timediff, bucketize, and then create a one hot vector, and then combine all three
#question: does order matter

df_onehot_comb = df_onehot_comb.withColumn('from_timestamp',to_timestamp(col('parent_timestamp')))\
  .withColumn('end_timestamp', to_timestamp(col('child_timestamp')))\
  .withColumn('DiffInSeconds',col("end_timestamp").cast("long") - col('from_timestamp').cast("long")) 
                                           

In [ ]:
# Use QuantileDiscretizer to calculate the bin boundaries
quantile_discretizer = QuantileDiscretizer(numBuckets=5, inputCol="DiffInSeconds", outputCol="binned_diff")
df_onehot_comb = quantile_discretizer.fit(df_onehot_comb).transform(df_onehot_comb)

# Convert the binned column to a sparse vector -> may want to consider a different method for this.
sparse_vectors = df_onehot_comb.rdd.map(lambda x: (x[0], Vectors.sparse(5, {x[2]: 1.0})))
sparse_df = spark.createDataFrame(sparse_vectors, ["id", "sparse_vector"])

# Show the resulting sparse vectors
sparse_df.show()